## Random agents with stepped vault APR

### Step 1: Setup experiment parameters

In [ ]:
from elfpy import DEFAULT_LOG_MAXBYTES

log_filename = "../../.logging/random_agent_demo.log" # Output filename for logging
config_file = "../../config/example_config.toml" # Config file to start from (overrides are specified below)

log_level = "WARNING" # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
max_bytes = DEFAULT_LOG_MAXBYTES # Maximum log file output size, in bytes. More than 100 files will cause overwrites.
pricing_model = "Hyperdrive" # can be yieldspace or hyperdrive

num_trading_days = 365 # Number of simulated trading days
blocks_per_day = 720 # Blocks in a given day (7200 means ~12 sec per block)
target_liquidity = 1e7 # target total liquidity of the initial market, before any trades
target_pool_apr = 0.05 # target pool APR of the initial market after the LP
fee_percent = 0.10 # fee percent collected on trades

num_agents = 5 # int specifying how many agents you want to simulate
agent_budget = 10_000 # max money an agent can spend
trade_chance = 5 / blocks_per_day # on a given block, an agent will trade with probability `trade_chance`

vault_apr_init = 0.01 # Initial vault APR
vault_apr_jump_size = 0.06 # Scale of the vault APR change (vault_apr (+/-)= jump_size)
vault_jumps_per_year = 4 # The average number of jumps per year
vault_apr_jump_direction = "up" # The direction of a rate change. Can be 'up', 'down', or 'random'.
vault_apr_lower_bound = 0.0 # minimum allowable vault apr
vault_apr_upper_bound = 0.6 # maximum allowable vault apr

### Step 2: Setup random agent

In [ ]:
from numpy.random._generator import Generator
import numpy as np
from elfpy.types import MarketActionType, MarketAction, WEI
from elfpy.agent import Agent
from elfpy.markets import Market

class RandomAgent(Agent):
    """
    Agent that randomly opens or closes longs or shorts
    """

    def __init__(self, rng: Generator, trade_chance: float, wallet_address: int, budget: int = 10_000) -> None:
        """Add custom stuff then call basic policy init"""
        self.trade_chance = trade_chance
        self.rng = rng
        super().__init__(wallet_address, budget)

    def action(self, market: Market) -> list[MarketAction]:
        """Implement a random user strategy

        The agent performs one of four possible trades:
            [OPEN_LONG, OPEN_SHORT, CLOSE_LONG, CLOSE_SHORT]
            with the condition that close actions can only be performed after open actions

        The amount opened and closed is random, within constraints given by agent budget & market reserve levels

        Parameters
        ----------
        market : Market
            the trading market

        Returns
        -------
        action_list : list[MarketAction]
        """
        gonna_trade = self.rng.choice([True, False], p=[self.trade_chance, 1-self.trade_chance])
        if not gonna_trade:
            return []
        # User can always open a trade, and can close a trade if one is open
        available_actions = [MarketActionType.OPEN_SHORT, MarketActionType.OPEN_LONG]
        has_opened_short = bool(any((short.balance > 0 for short in self.wallet.shorts.values())))
        if has_opened_short:
            available_actions.append(MarketActionType.CLOSE_SHORT)
        has_opened_long = bool(any((long.balance > 0 for long in self.wallet.longs.values())))
        if has_opened_long:
            available_actions.append(MarketActionType.CLOSE_LONG)
        action_type = self.rng.choice(available_actions, size=1) # choose one random trade type
        # trade amount is also randomly chosen
        match action_type:
            case MarketActionType.OPEN_SHORT:
                random_normal = self.rng.normal(loc=self.budget * 0.1, scale=self.budget * 0.01)
                max_short = self.get_max_short(market)
                if max_short > WEI: # if max_short is greater than the minimum eth amount
                    trade_amount = np.maximum(WEI, np.minimum(max_short, random_normal)) # WEI <= trade_amount <= max_short
                    action_list = [
                        self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=market.time),
                    ]
                else: # no short is possible
                    action_list = []
            case MarketActionType.OPEN_LONG:
                random_normal = self.rng.normal(loc=self.budget * 0.1, scale=self.budget * 0.01)
                max_long = self.get_max_long(market)
                if max_long > WEI: # if max_long is greater than the minimum eth amount
                    trade_amount = np.maximum(WEI, np.minimum(max_long, random_normal))
                    action_list = [
                        self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=market.time),
                    ]
                else:
                    action_list = []
            case MarketActionType.CLOSE_SHORT:
                short_time = self.rng.choice(list(self.wallet.shorts))
                trade_amount = self.wallet.shorts[short_time].balance # close the full trade
                action_list = [
                    self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=short_time),
                ]
            case MarketActionType.CLOSE_LONG:
                long_time = self.rng.choice(list(self.wallet.longs))
                trade_amount = self.wallet.longs[long_time].balance # close the full trade
                action_list = [
                    self.create_agent_action(action_type=action_type, trade_amount=trade_amount, mint_time=long_time),
                ]
        return action_list


def get_example_agents(rng: Generator, budget: float, new_agents: int, existing_agents: int = 0) -> list[Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + new_agents):
        agent = RandomAgent(
            rng=rng,
            trade_chance=trade_chance,
            wallet_address=address,
            budget=budget,
        )
        agent.log_status_report()
        agents += [agent]
    return agents

### Step 3: Define vault apr process

In [ ]:
from scipy import special
def homogeneous_poisson(
    rng: Generator, rate: float, tmax: int, bin_size: int = 1
    ) -> np.ndarray:
    """Generate samples from a homogeneous Poisson distribution

    Attributes
    ----------
    rng: np.random.Generator
        random number generator with preset seed
    rate: float
        number of events per time interval (units of 1/days)
    tmax: float
        total number of days (units of days; sets distribution support)
    bin_size: float
        resolution of the simulation
    """
    nbins = np.floor(tmax/bin_size).astype(int)
    prob_of_spike = rate * bin_size
    events = (rng.random(nbins) < prob_of_spike).astype(int)
    return events

def event_generator(rng, n_trials, rate, tmax, bin_size):
    """Generate samples from the poisson distribution"""
    for i in range(n_trials):
        yield homogeneous_poisson(rng, rate, tmax, bin_size)

def poisson_prob(k, lam):
    """https://en.wikipedia.org/wiki/Poisson_distribution"""
    return lam**k/special.factorial(k)*np.exp(-lam)


In [ ]:
import matplotlib.pyplot as plt
import elfpy.utils.parse_config as config_utils
from elfpy.utils.outputs import get_gridspec_subplots

tmp_config = config_utils.load_and_parse_config_file(config_file)
rng = tmp_config.simulator.rng
n_trials = 1
avg_events_per_interval = 4 # 1 event every 3 months = 4 events per year; same as vault_jumps_per_year
num_bins = 365 # days in a year
bin_size = 1
rate = avg_events_per_interval / num_bins
tmax = num_bins
events_poisson = list(event_generator(rng, n_trials, rate, tmax, bin_size))[0]
time = np.arange(len(events_poisson))

n_trials = 100000
n_events = np.array([np.sum(events) for events in event_generator(rng, n_trials, rate, tmax, bin_size)])
bin_edges =  np.arange(n_events.max() + 1) - 0.5
lam = rate * tmax
k = bin_edges + 0.5
prob = poisson_prob(k, lam)

fig, axs, gridspec = get_gridspec_subplots(nrows=2, ncols=1, hspace=0.5)

axs[0].plot(time, events_poisson)
axs[0].set_title("events")
axs[0].set_xlabel("time (days)")
axs[0].set_yticks([0, 1])

axs[1].hist(n_events, bin_edges, density=True, fc="none", ec="k")
axs[1].plot(k, prob, c="b")
axs[1].set_title("event probability")
axs[1].set_xlabel(f"number of events in {tmax} days")
axs[1].set_ylabel(f"probability")

In [ ]:
def vault_flip_probs(apr: float, min_apr: float=0.0, max_apr: float=1.0, num_points: int = int(1e9)):
    """
    probability of going up is 1 when apr is min
    probability of going down is 1 when apr is max
    probability is 0.5 either way when apr is half way between max and min
    """
    aprs = np.linspace(min_apr, max_apr, num=num_points)
    get_index = lambda value, array : (np.abs(array - value)).argmin()
    apr_index = get_index(apr, aprs) # return whatever value in aprs array that apr is closest to
    up_probs = np.linspace(1, 0, num=num_points)
    up_prob = up_probs[apr_index]
    down_prob = 1 - up_prob
    return (down_prob, up_prob)

min_apr = 0.2
max_apr = 0.8
num_points = 100
aprs = np.linspace(min_apr, max_apr, num=num_points)
vault_flip_probs(aprs[0])
#down_probs = [vault_flip_probs(apr)[0] for apr in aprs]
#up_probs = [vault_flip_probs(apr)[1] for apr in aprs]
#fig, axs, gs = get_gridspec_subplots()
#ax = axs[0]
#ax.plot(aprs, down_probs, c='b', label="prob(going down)")
#ax.plot(aprs, up_probs, c='r', label="prob(going up)")

In [ ]:
from elfpy.utils.config import Config


def poisson_vault_apr(
    config: Config,
    initial_apr: float,
    jump_size: float,
    vault_jumps_per_year: int,
    direction: str,
    lower_bound: float = 0.0,
    upper_bound: float = 1.0,
) -> Generator:
    # vault rate changes happen once every vault_jumps_per_year, on average
    num_bins = 365
    bin_size = 1
    rate = vault_jumps_per_year / num_bins
    tmax = num_bins
    do_jump = homogeneous_poisson(config.simulator.rng, rate, tmax, bin_size)
    vault_apr = np.array([initial_apr] * config.simulator.num_trading_days)
    for day in range(config.simulator.num_trading_days):
        if do_jump[day]:
            match direction:
                case "up":
                    sign = 1
                case "down":
                    sign = -1
                case "random":
                    l_pdf = np.linspace(start=lower_bound, stop=upper_bound, num=int(np.floor(config.simulator.num_trading_days/2)))
                    r_pdf = np.linspace(start=upper_bound, stop=lower_bound, num=int(np.ceil(config.simulator.num_trading_days/2)))
                    pdfs = np.concatenate((l_pdf, r_pdf))
                    choices = [-1]*len(l_pdf) + [1]*len(r_pdf)
                    sign = config.simulator.rng.choice([-1, 1], size=1).item() # flip a coin
                case _:
                    raise ValueError(f"Direction must be 'up', 'down', or 'random'; not {direction}")
            step = sign * jump_size
            apr = np.minimum(upper_bound, np.maximum(lower_bound, vault_apr[day] + step))
            vault_apr[day:] = apr
    for apr in vault_apr:
        yield apr

import matplotlib.pyplot as plt
import elfpy.utils.parse_config as config_utils
override_dict = {
    "pricing_model_name": pricing_model,
    "num_trading_days": num_trading_days,
    "num_blocks_per_day": blocks_per_day,
    "pricing_model_name": pricing_model,
    "target_liquidity": target_liquidity,
    "target_pool_apr": target_pool_apr,
    "fee_percent": fee_percent,
    "logging_level": log_level,
}
config = config_utils.override_config_variables(config_utils.load_and_parse_config_file(config_file), override_dict)

vault_apr = list(poisson_vault_apr(
    config=config,
    initial_apr=vault_apr_init,
    jump_size=vault_apr_jump_size,
    vault_jumps_per_year=vault_jumps_per_year,
    direction=vault_apr_jump_direction,
))

fig, axs, gridspec = get_gridspec_subplots()
ax = axs[0]
ax.plot(np.arange(config.simulator.num_trading_days), vault_apr, c='k')
ax.set_xlabel("Time (days)")
ax.set_ylabel("Vault APR")

### Step 4: Setup experiment configuration using parameters specified above

In [ ]:
import elfpy.utils.parse_config as config_utils

# parameters set at the top of the notebook
override_dict = {
    "pricing_model_name": pricing_model,
    "num_trading_days": num_trading_days,
    "num_blocks_per_day": blocks_per_day,
    "pricing_model_name": pricing_model,
    "target_liquidity": target_liquidity,
    "target_pool_apr": target_pool_apr,
    "fee_percent": fee_percent,
    "logging_level": log_level,
}
config = config_utils.override_config_variables(config_utils.load_and_parse_config_file(config_file), override_dict)

# override the vault_apr, which is based on some variables set above
override_dict["vault_apr"] = lambda: poisson_vault_apr(
    config=config,
    initial_apr=vault_apr_init,
    jump_size=vault_apr_jump_size,
    vault_jumps_per_year=vault_jumps_per_year,
    direction=vault_apr_jump_direction,
    lower_bound=vault_apr_lower_bound,
    upper_bound=vault_apr_upper_bound,
)
config = config_utils.override_config_variables(config, override_dict)

### Step 5: Run the simulation

In [ ]:
from elfpy.utils import sim_utils
import elfpy.utils.outputs as output_utils

# define root logging parameters
output_utils.setup_logging(
    log_filename=log_filename,
    max_bytes=max_bytes,
    log_level=config_utils.text_to_logging_level(config.simulator.logging_level),
)

# initialize the simulator
random_agents = get_example_agents(rng=config.simulator.rng, budget=agent_budget, new_agents=num_agents, existing_agents=1)
simulator = sim_utils.get_simulator(config, random_agents)

# run the simulation
simulator.run_simulation()

### Step 6: Plot simulation results

In [ ]:
import elfpy.utils.post_processing as post_processing
trades = post_processing.compute_derived_variables(simulator)

In [ ]:
import logging
import matplotlib.pyplot as plt

logging.getLogger().setLevel(logging.WARNING)  # events of this level and above will be tracked

vault_spot_size = 10
spot_colors = ['blue', 'black']
fig, ax = plt.subplots()
x_data = trades.day
ax.plot(x_data, trades.vault_apr, label="Vault", c=spot_colors[0])
#ax.scatter(x_data, trades.vault_apr, label="Vault", s=vault_spot_size, c=spot_colors[0])
prev_apr = trades.loc[trades.run_trade_number==0].pool_apr
prev_time = 0
for day in set(x_data):
    trade_numbers = trades.loc[trades.day==day].run_trade_number
    spot_sizes = np.linspace(0.2, 0.9, len(trade_numbers))
    for trade_idx, trade_number in enumerate(trade_numbers):
        pool_apr = trades.loc[trades.run_trade_number==trade_number].pool_apr
        pool_spot_size = vault_spot_size * 0.5 #spot_sizes[trade_idx]
        time = day + spot_sizes[trade_idx]
        if day == 0 and trade_idx == len(trade_numbers)-1: # only label the first plot
            ax.plot(time, pool_apr, label="Pool", c=spot_colors[1])
            #ax.scatter(time, pool_apr, label="Pool", s=pool_spot_size, c=spot_colors[1])
        else:
            ax.plot(time, pool_apr, c=spot_colors[1])
            #ax.scatter(time, pool_apr, s=pool_spot_size, c=spot_colors[1])
        ax.plot([prev_time, time], [prev_apr, pool_apr], color='k', linestyle='-', linewidth=0.1)
        prev_time = time
        prev_apr = pool_apr
ax.set_xlabel("Day")
ax.set_ylabel("APR")
plt.legend()
xtick_step = 50
ax.set_xticks([x for x in range(0, simulator.config.simulator.num_trading_days + 1, xtick_step)])
ax.set_xticklabels([str(x+1) for x in range(0, simulator.config.simulator.num_trading_days + 1, xtick_step)])
ax.set_title("market pool and underlying vault APR")
plt.grid()

In [ ]:
fig = output_utils.plot_wallet_returns(simulator, exclude_first_agent=True, xtick_step=2000)

In [ ]:
fig = output_utils.plot_pool_apr(simulator)

In [ ]:
fig = output_utils.plot_market_spot_price(simulator)

In [ ]:
fig = output_utils.plot_market_lp_reserves(simulator)

In [ ]:
fig = output_utils.plot_longs_and_shorts(simulator, xtick_step=2000)